# Apply FS in ANN
Condition of improvement only if RMSE on spike regions improves

In [1]:
! pip install pandas
! pip install numpy
! pip install sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []

# Import data & treat it

In [2]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)


# Import libraries:

In [3]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers

# parameters for ANN
splits = 7
epochs = 80

# divide into features and label
X = data.iloc[:, 0:14]
y = data.loc[:, 'Offers']

Using TensorFlow backend.


# Do first prediciton with first feature in list from Linear Regression FS:

In [4]:
# data set to append here (start with first feature added)
X_ = X.loc[:,'PrevDay']

# do first prediction
X_.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X_ = X_.astype('float64')
X_ = X_.round(20)

X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)

X_train = np.array(X_train)
X_train = X_train.reshape(-1, 1)
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 1)

sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 1))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.3))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = splits)    
regressor = regressor_tunning()

# train model
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    regressor.fit(X_train_split, y_train_split,  
                         shuffle = False, 
                         validation_split = 0.2,
                         batch_size = 20, 
                         epochs = epochs)

# make predictions and evaluate for all regions
y_pred = regressor.predict(X_test)

# =============================================================================
# METRICS EVALUATION (1) for the whole test set
# =============================================================================
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

# calculate metrics
rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)

# append to list
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)

# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# download spike indication binary set
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)

# append ot lists
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)

# append to list
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 7s 15ms/step - loss: 13340.4484 - mse: 13340.4482 - mae: 109.9549 - val_loss: 34637.3145 - val_mse: 34637.3125 - val_mae: 132.8003
Epoch 2/80
500/500 [==============================] - 0s 578us/step - loss: 13219.1904 - mse: 13219.1904 - mae: 109.4139 - val_loss: 34394.4432 - val_mse: 34394.4453 - val_mae: 131.8861
Epoch 3/80
500/500 [==============================] - 0s 580us/step - loss: 12872.0937 - mse: 12872.0928 - mae: 107.8353 - val_loss: 33702.5513 - val_mse: 33702.5508 - val_mae: 129.2464
Epoch 4/80
500/500 [==============================] - 0s 600us/step - loss: 11855.7543 - mse: 11855.7568 - mae: 102.9922 - val_loss: 31842.5462 - val_mse: 31842.5469 - val_mae: 121.8672
Epoch 5/80
500/500 [==============================] - 0s 578us/step - loss: 9493.5852 - mse: 9493.5859 - mae: 90.5908 - val_loss: 27412.1698 - val_mse: 27412.1699 - val_mae: 102.1680
Epoch 6/80
500/500 [========

# Apply loop for the rest of the list of features with condition of improvement:

In [5]:
# features list; order made according to Linear Regression FS
features_list = ['APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

best_score = rmse_spike

for i in features_list: 
    
   # X_recovery = X_.copy()
    X_recovery = X_
    
    X_ = pd.concat([X_, X.loc[:,i]], axis = 1)
                    
    X_.fillna(method = 'ffill', inplace = True)
    y.fillna(method = 'ffill', inplace = True)

    X_ = X_.astype('float64')
    X_ = X_.round(20)

    X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)
    
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)

    # possible debug
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = len(X_.columns)))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
        for layer in range(n_hidden):
            model.add(Dense(units = n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.3))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = 0.001)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = splits)    
    regressor = regressor_tunning()

    # train model
    for train_index, test_index in tscv.split(X_train):
        X_train_split, X_test_split = X_train[train_index], X_train[test_index]
        y_train_split, y_test_split = y_train[train_index], y_train[test_index]
        regressor.fit(X_train_split, y_train_split,  
                             shuffle = False, 
                             validation_split = 0.2,
                             batch_size = 20, 
                             epochs = epochs)

    # make predictions and evaluate for all regions
    y_pred = regressor.predict(X_test)

    # =============================================================================
    # METRICS EVALUATION (1) for the whole test set
    # =============================================================================
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    # calculate metrics
    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)

    # append to list
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # METRICS EVALUATION (2) on spike regions
    # =============================================================================

    # download spike indication binary set
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)

    # append ot lists
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # METRIC EVALUATION (3) on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)

    # append to list
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

    # condition of improvement for FS
    if best_score < rmse_spi[-1]:
        X_ = X_recovery
    else:
        X_ = X_
        best_score = rmse_spike


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 4s 7ms/step - loss: 13306.2625 - mse: 13306.2617 - mae: 109.8038 - val_loss: 34546.6887 - val_mse: 34546.6836 - val_mae: 132.4607
Epoch 2/80
500/500 [==============================] - 0s 517us/step - loss: 13054.2635 - mse: 13054.2617 - mae: 108.6513 - val_loss: 34028.3438 - val_mse: 34028.3438 - val_mae: 130.4986
Epoch 3/80
500/500 [==============================] - 0s 572us/step - loss: 12282.1594 - mse: 12282.1592 - mae: 105.0098 - val_loss: 32508.2935 - val_mse: 32508.2949 - val_mae: 124.5681
Epoch 4/80
500/500 [==============================] - 0s 568us/step - loss: 10174.6887 - mse: 10174.6904 - mae: 94.2229 - val_loss: 28468.0892 - val_mse: 28468.0898 - val_mae: 107.2245
Epoch 5/80
500/500 [==============================] - 0s 637us/step - loss: 5962.1595 - mse: 5962.1597 - mae: 65.7649 - val_loss: 20759.1571 - val_mse: 20759.1562 - val_mae: 61.8548
Epoch 6/80
500/500 [===========

In [6]:
features_list_index =   ['PrevDay',
                         'APXP', 
                         'LOLP',  
                         'In_gen',
                         'Ren_R',
                         'DA_imb_France', 
                         'Rene',
                         'ratio_offers_vol',
                         'DA_price_france',
                         'TSDF',
                         'dino_bin',
                         'DA_margin',
                         'Im_Pr']

results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,}, index = features_list_index)

results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
PrevDay,30.859331,21.799345,71.579277,60.584728,18.356216,16.077741
APXP,30.301084,21.129774,69.999551,58.298618,18.186944,15.646642
LOLP,30.058745,20.207957,70.889924,58.963571,17.189361,14.490745
In_gen,30.268033,21.076997,70.140771,58.516779,18.042957,15.553896
Ren_R,30.546874,20.973736,69.971366,57.481208,18.668397,15.588169
DA_imb_France,30.410576,20.735486,70.109970,57.701602,18.333324,15.282260
Rene,30.883907,21.845374,69.186281,56.949086,19.707463,16.666889
ratio_offers_vol,31.161971,21.585353,71.456593,59.024224,19.002064,16.062387
DA_price_france,30.593621,21.112575,69.783611,57.464252,18.858940,15.749991
TSDF,30.744812,22.015641,67.822606,55.600419,20.152152,17.061229


In [7]:
X_.columns

Index(['PrevDay', 'APXP', 'Ren_R', 'Rene', 'TSDF'], dtype='object')

In [8]:
best_score

67.82260572023111